# Introduction
This notebook constructs a simplified metabolic model of Rhodospirillum rubrum 


In [28]:
import numpy as np
import pandas as pd
import subprocess
import re
import os
import sys
import cobra
from importlib import reload
from IPython.core.display import display
pd.set_option('display.max_columns', None,'display.max_rows', None)
pd.set_option('display.max_colwidth',None)
pd.options.mode.chained_assignment = None


sys.path.append('../')
import utils as utils


## Set standard Parameters
T = 298.15
R = 8.3144598e-03
RT = R*T
N_avogadro = 6.022140857e+23
VolCell = 1.0e-15
Concentration2Count = N_avogadro * VolCell
concentration_increment = 1/(N_avogadro*VolCell)


# Set path to the directory that contains the model files
mf_dir = 'model_files/'

/var/folders/45/hnmrvf5d6wdf1h8m7d4cn5s4hp2pl3/T/ipykernel_26744/1931947951.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


### Create a stoichiometric matrix

In [29]:
left ='LEFT'
right = 'RIGHT'
left_compartment = 'LEFT_COMPARTMENT'
right_compartment = 'RIGHT_COMPARTMENT'
enzyme_level = 'ENZYME_LEVEL'
deltag0 = 'DGZERO'
deltag0_sigma = 'DGZERO StdDev'
same_compartment = 'Same Compartment?'
full_rxn = 'Full Rxn'
rxn = 'REACTION'
flux = 'Flux'

reactions = pd.DataFrame(index=[],columns=[left, right, left_compartment, right_compartment, enzyme_level, deltag0, deltag0_sigma, same_compartment,full_rxn])
reactions.index.name='REACTION'
S_matrix = pd.DataFrame(index=[],columns=[enzyme_level])
S_matrix.index.name='REACTION'

### Read In Pathway Files

In [30]:
rxn_file = 'Rpal_Glycolysis-CBB-PPP-TCA-GlyoxylateCycle-Ethanol-Methionine-SAMcycles-SerCys.txt'
reactions = pd.DataFrame(index=[],columns=[left, right, left_compartment, right_compartment, enzyme_level, deltag0, deltag0_sigma, same_compartment,full_rxn])
reactions.index.name='REACTION'
S_matrix = pd.DataFrame(index=[],columns=[enzyme_level])
S_matrix.index.name='REACTION'

[reactions, S_matrix] = utils.ReadRxnFrameFile(mf_dir + rxn_file,reactions, S_matrix)

#### Reductive TCA Cycle, Glyoxylate Shunt and Fe-S Pyruvate Dehydrogenase

In [31]:
import logging

logging.getLogger("cobra").setLevel(logging.ERROR)


rxn_file = 'reductive_TCAcycle_glyoxylate_shunt_FeS_PDH.sbml'
[reactions, S_matrix] = utils.ReadSBMLFile(mf_dir + rxn_file,reactions, S_matrix)

ERROR:cobra.io.sbml:No objective coefficients in model. Unclear what should be optimized


In [32]:
rxn_file = 'superpathway_branched_chain_aa.txt'
[reactions, S_matrix] = utils.ReadRxnFrameFile(mf_dir + rxn_file,reactions, S_matrix)

In [33]:
rxn_file = 'Gly-Trp-Tyr-His--Lys-Thr-Chorismate-PRPP.txt'
[reactions, S_matrix] = utils.ReadRxnFrameFile(mf_dir + rxn_file,reactions, S_matrix)

In [34]:
rxn_file = 'rubrum_phenylalanine_synthesis.txt'
[reactions, S_matrix] = utils.ReadRxnFrameFile(mf_dir + rxn_file,reactions, S_matrix)

In [35]:
rxn_file = 'alanine_biosynthesis.txt'
[reactions, S_matrix] = utils.ReadRxnFrameFile(mf_dir + rxn_file,reactions, S_matrix)

In [36]:
rxn_file = 'asparagine_synthesis.txt'
[reactions, S_matrix] = utils.ReadRxnFrameFile(mf_dir + rxn_file,reactions, S_matrix)

#### Arginine synthesis

In [37]:
rxn_file = 'arginine_biosynthesisII_rubrum.sbml'
[reactions, S_matrix] = utils.ReadSBMLFile(mf_dir + rxn_file,reactions, S_matrix)
rxn_file = 'carbonic_anhydrase.sbml'
[reactions, S_matrix] = utils.ReadSBMLFile(mf_dir + rxn_file,reactions, S_matrix)
rxn_file = 'proline_synthesis_rubrum.sbml'
[reactions, S_matrix] = utils.ReadSBMLFile(mf_dir + rxn_file,reactions, S_matrix)

ERROR:cobra.io.sbml:No objective coefficients in model. Unclear what should be optimized
ERROR:cobra.io.sbml:No objective coefficients in model. Unclear what should be optimized
ERROR:cobra.io.sbml:No objective coefficients in model. Unclear what should be optimized


In [38]:
rxn_file = 'sulfate_assimilation_rpal.txt'
[reactions, S_matrix] = utils.ReadRxnFrameFile(mf_dir + rxn_file,reactions, S_matrix)

#### Malic Reactions and PRPP synthesis

In [39]:
rxn_file = 'malic_rxns_rubrum.sbml'
[reactions, S_matrix] = utils.ReadSBMLFile(mf_dir + rxn_file,reactions, S_matrix)
rxn_file = 'PRPP_synthesis_rubrum.sbml'
[reactions, S_matrix] = utils.ReadSBMLFile(mf_dir + rxn_file,reactions, S_matrix)

ERROR:cobra.io.sbml:No objective coefficients in model. Unclear what should be optimized
ERROR:cobra.io.sbml:No objective coefficients in model. Unclear what should be optimized


#### Purine and Pyrimidine Synthesis

In [40]:
rxn_file = 'pyrimidine_biosynthesis_rubrum.sbml'
[reactions, S_matrix] = utils.ReadSBMLFile(mf_dir + rxn_file,reactions, S_matrix)
rxn_file = 'purine_biosynthesis_rubrum.sbml'
[reactions, S_matrix] = utils.ReadSBMLFile(mf_dir + rxn_file,reactions, S_matrix)

ERROR:cobra.io.sbml:No objective coefficients in model. Unclear what should be optimized
ERROR:cobra.io.sbml:No objective coefficients in model. Unclear what should be optimized


### Turn some reaction pathways off

In [41]:
# Glucose 6 phosphate => fructose-6-phosphate rxn:
reactions.loc['PGLUCISOM-RXN', enzyme_level] = np.float64(0.0);
S_matrix.loc['PGLUCISOM-RXN', enzyme_level] = np.float64(0.0);

# phophofructokinase reaction        
reactions.loc['6PFRUCTPHOS-RXN', enzyme_level] = np.float64(0.0);
S_matrix.loc['6PFRUCTPHOS-RXN', enzyme_level] = np.float64(0.0);

# Rxn 3-phospho-D-glyceroyl-phosphate + NADH + H+ = D-glyceraldehyde 3-phosphate + NAD+ + phosphate
reactions.loc['GAPOXNPHOSPHN-RXN', enzyme_level] = np.float64(0.0);
S_matrix.loc['GAPOXNPHOSPHN-RXN', enzyme_level] = np.float64(0.0);

# Transaldolase reaction
reactions.loc['TRANSALDOL-RXN', enzyme_level] = np.float64(0.0);
S_matrix.loc['TRANSALDOL-RXN', enzyme_level] = np.float64(0.0);

# Drop Generic Reaction RXN-12912
reactions.drop('RXN-12912',axis = 0,inplace=True)
S_matrix.drop('RXN-12912',axis = 0,inplace=True)

# Phosphoglycerate isomerase reaction
reactions.loc['RXN-15513', enzyme_level] = np.float64(0.0);
S_matrix.loc['RXN-15513', enzyme_level] = np.float64(0.0);
reactions.loc['MALATE-DEH-RXN', enzyme_level] = np.float64(0.0);
S_matrix.loc['MALATE-DEH-RXN', enzyme_level] = np.float64(0.0);
reactions.loc['ISOCIT-CLEAV-RXN', enzyme_level] = np.float64(0.0);
S_matrix.loc['ISOCIT-CLEAV-RXN', enzyme_level] = np.float64(0.0);



# Oxidative  TCA cycle reactions
reactions.loc['PDH', enzyme_level] = np.float64(0.0);
S_matrix.loc['PDH', enzyme_level] = np.float64(0.0);
reactions.loc['2OXOGLUTARATEDEH-RXN', enzyme_level] = np.float64(0.0); #differs
S_matrix.loc['2OXOGLUTARATEDEH-RXN', enzyme_level] = np.float64(0.0); #differs


S_matrix.fillna(0,inplace=True)
S_active = S_matrix[S_matrix[enzyme_level] > 0.0]
display(S_active.shape)
active_reactions = reactions[reactions[enzyme_level] > 0.0]
del S_active[enzyme_level]

# Delete any columns/metabolites that have all zeros in the S matrix:
S_active = S_active.loc[:, (S_active != 0).any(axis=0)]
#display(S_active)
reactions[left_compartment] = 'CYTOPLASM'
reactions[right_compartment] = 'CYTOPLASM'
#display(reactions)

(174, 195)

### Ethylene Pathway

In [42]:
rxn_file = 'ethylene_pwy.dat'

[reactions, S_active] = utils.ReadRxnFile(mf_dir + rxn_file,reactions, S_active)
reactions[left_compartment] = 'CYTOPLASM'
reactions[right_compartment] = 'CYTOPLASM'
reactions[same_compartment] = 1.0
S_active.fillna(0,inplace=True)
del S_active[enzyme_level]
#display(reactions)

In [43]:
reactions.loc['SAM Hydrolase::ADENOSYLMETHIONINE-HYDROLASE-RXN',deltag0] = np.array([-8.11])[0]
reactions.loc['MTA nucleosidase::METHYLTHIOADENOSINE-NUCLEOSIDASE-RXN',deltag0] = np.array([-8.11])[0]
reactions.loc['MTNA::5.3.1.23-RXN',deltag0] = np.array([-8.11])[0]
reactions.loc['MT-Ribulose lyase::MT-Ribulose lyase',deltag0] = np.array([-8.11])[0]
reactions.loc['MT-acetate reductase::MT-acetate reductase',deltag0] = np.array([-8.11])[0]
reactions.loc['MT-ethanol dehydrogenase::MT-ethanol dehydrogenase',deltag0] = np.array([-8.11])[0]
reactions.loc['O-ACETYLHOMOSERINE-THIOL-LYASE::O-ACETYLHOMOSERINE-THIOL-LYASE-RXN',deltag0] = np.array([-8.11])[0]

## Calculate Standard Free Energies of Reaction 

Some reactions are set manually because they are not found in eQuilibrator, this may be resolved by future releases

In [44]:
reactions.loc['RXN-15121',deltag0] = np.array([-17.0])[0]
reactions.loc['RXN-15123',deltag0] = np.array([-17.0])[0]

### Overall Thermodynamics of SAM Cycles I and II

ATP consumption provides the carbon for driving SAM cycles.

**SAM cycle I overall reaction:**
$$
\text{S-adenosyl-L-methionine + H2O}  \longrightarrow  

\text{adenine + (4S)-4,5-dihydroxypentan-2,3-dione + L-methionine} \\  

\Delta G^{\circ} = -16.57 \pm 100,000 \text{kJ/mol}

$$

**SAM cycle II overall reaction:**
$$
\text{S-adenosyl-L-methionine + H2O}  \longrightarrow 

\text{adeninosine + L-methionine} \\ 

\Delta G^{\circ} = +45.14 \pm 100,000 \text{kJ/mol}
$$

The free energy change for the last reaction can be understood in terms of component reactions,

$$
\text{S-adenosyl-L-methionine + phosphate + diphosphate} \longrightarrow 
\text{L-Methionine + ATP + H2O}\\ 
\Delta G^{\circ} = +93.96 \pm 100,000 \text{kJ/mol} \\
\text{ADP + H2O}  \longrightarrow 
\text{adeninosine + diphosphate} \\ 
 \Delta G^{\circ} = -22.7 \pm 2.9 \text{kJ/mol} \\
\text{ATP + H2O}  \longrightarrow 
\text{ADP + phosphate} \\ 
 \Delta G^{\circ} = -26.1 \pm 0.6 \text{kJ/mol} \\

\text{S-adenosyl-L-methionine + H2O} \longrightarrow  \text{L-methionine + adenosine} \\
 \Delta G_{total}^{\circ} = +45.16
$$


### Overall Thermodynamics of Methylthio-Alkane Reductase/Ethylene Synthesis Pathway
The standard free energies of the specific reactions of this pathway are not available. But we can calculate the overall standard free energy change by breaking the overall reaction down into subreactions that can be calculated.
Also, we can't judge whether this pathway is feasible until correct free energies are included.


The overall reaction that combines the seven reactions is,

$$
\text{S-adenosyl-L-methionine +  orthophosphate + NADH} \\
\text{ + 2 Fd-red + O-acetyl-L-homoserine}  \longrightarrow \\
\text{L-homoserine + adenine + NAD+ + ethylene  + H2 + 2 Fd-ox + L-methionine + acetate +  glycerone phoshpate} \\ 
\Delta G_{overall} = -56.8 kJ/mol
$$

To calculate the overall free energy change, this reaction was broken down into five smaller reactions:
$$
\text{2 Ferredoxin-reduced + NAD+}  \longrightarrow  2 \text{Ferredoxin-oxidized + NADH} \\
 \Delta G^{\circ} = 2*( -8.5 kJ/mol) ~ eQuilibrator \\
\text{O-acetyl-L-homoserine + H2O}  \longrightarrow  \text{L-homoserine + acetate}  \\
\Delta G^{\circ} = -9.5 kJ/mol ~ eQuilibrator\\
\text{Ribose-5-phosphate + 2 NADH}  \longrightarrow  \text{Ethylene + Glycerone phosphate + 2 NAD + 2 H2O} \\
\Delta G^{\circ} = -58.9 kJ/mol ~  eQuilibrator \\
\text{Ribose + orthosphosphate}  \longrightarrow  \text{ribose-5-phosphate + H2O} \\
 \Delta G^{\circ} = 1.4 kJ/mol ~ eQuilibrator \\
\text{S-adenosyl-l-methionine + 2 H2O}  \longrightarrow \text{adenine + ribose + methionine}  \Delta G^{\circ} = 39.8 kJ/mol \text{(see below)} \\
 \Delta G_{overall} = -56.8 kJ/mol
$$

The last reaction,
$$
\text{S-adenosyl-l-methionine + 2 H2O}  \longrightarrow  \text{adenine + ribose + methionine} 
$$

was broken down further because there is no eQuilibrator data:

$$
\text{Adenosine + H2O}  \longrightarrow  \text{adenine + ribose}   \\
\Delta G^{\circ} = -4.9 kJ/mol  \\
\text{S-adenosyl-L-methionine + H2O}  \longrightarrow  \text{adenosine + methionine} \\
$$

The last reaction is assumed to have the same $\Delta G^{\circ}$ as:
$$
\text{S-Ribosyl-L-homocysteine + H2O}  \longrightarrow  \text{Homocysteine + Ribose} \\
 \Delta G^{\circ} = 44.7 kJ/mol  \\
 \text{total } \Delta G^{\circ} = 39.8 kJ/mol
$$


Until we get standard free energy changes for individual reactions, we will assume that each reaction has a free energy change of 
$\Delta G_{i}^{\circ}=\Delta G_{overall}//7 = -8.11 kJ/mol$

In [45]:
reactions.loc['SAM Hydrolase::ADENOSYLMETHIONINE-HYDROLASE-RXN',deltag0] = np.array([-8.11])[0]
reactions.loc['MTA nucleosidase::METHYLTHIOADENOSINE-NUCLEOSIDASE-RXN',deltag0] = np.array([-8.11])[0]
reactions.loc['MTNA::5.3.1.23-RXN',deltag0] = np.array([-8.11])[0]
reactions.loc['MT-Ribulose lyase::MT-Ribulose lyase',deltag0] = np.array([-8.11])[0]
reactions.loc['MT-acetate reductase::MT-acetate reductase',deltag0] = np.array([-8.11])[0]
reactions.loc['MT-ethanol dehydrogenase::MT-ethanol dehydrogenase',deltag0] = np.array([-8.11])[0]
reactions.loc['O-ACETYLHOMOSERINE-THIOL-LYASE::O-ACETYLHOMOSERINE-THIOL-LYASE-RXN',deltag0] = np.array([-8.11])[0]

### Ferredoxin-Oxidoreductases for Reverse TCA Cycle

**Pyruvate Ferredoxin-Oxidoreductase**
(PYRUFLAVREDUCT-RXN)

| Reaction | Energy Change |
|:---:|---:|
| 2 FeDox(ox) + 2e-   => 2 FeDox(red)     | E0 ~ -2(0.400V)| 
|      H$_2$ => 2H+(aq) + 2 e−		          | E0 = -2(0. 0V) |
| |$\Delta G^{\circ}$ = =-z$F\Delta$ E0 = +2(96.485kJ/V)(0.400V) = 77.188 kJ/mol|
| PYRUVATE + COA = ACETYL-COA + CO2 + H$_2$ | $\Delta G^{\circ}$ = 1.0 kJ/mol |
|Sum Total:| |
|PYRUVATE + COA + 2.0 FeDox(ox) = ACETYL-COA + CO2 + 2.0 FeDox(red) | $\Delta G^{\circ}$ = 78.188 kJ/mol |


**2-Oxoglutarate Ferredoxin-Oxidoreductase**
(2-OXOGLUTARATE-SYNTHASE-RXN)

| Reaction | Energy Change |
|:---:|---:|
| 2 FeDox(ox) + 2e-   => 2 FeDox(red)     | E0 ~ -2(0.400V)| 
|      H$_2$ => 2H+(aq) + 2 e−		          | E0 = -2(0. 0V) |
| |$\Delta G^{\circ}$ = =-z$F\Delta$ E0 = +2(96.485kJ/V)(0.400V) = 77.188 kJ/mol|
| 2-OXOGLUTARATE + COA = SUCCINYL-COA + CO$_2$ + H$_{2}$ |$\Delta G^{\circ}$ = 9.5 kJ/mol |
| Sum Total:||
|2-OXOGLUTARATE + COA + 2.0 FeDox(ox) = SUCCINYL-COA + CO$_2$ + 2.0 FeDox(red) |$\Delta G^{\circ}$ = 86.688 kJ/mol |


In [46]:
reactions.loc['2-OXOGLUTARATE-SYNTHASE-RXN',deltag0] = np.array([86.688])[0]
reactions.loc['PYRUFLAVREDUCT-RXN',deltag0] = np.array([78.188])[0]

### Equilibrator Computations

Equilibrator_api Version 0.3.1

In [47]:
import sys
from equilibrator_api import ComponentContribution, Q_
cc = ComponentContribution()
import equilibrator_api
display(equilibrator_api.__version__)

print_output = False

file = mf_dir + 'temp.dg0'
free_energies = pd.read_csv(file,sep='\t',header=None, index_col=0,names=['dg0','dg0_sigma'])
for idx in free_energies.index:
    if idx in reactions.index:
        reactions.loc[idx,deltag0] = free_energies.loc[idx,'dg0']


eq_api = cc
cc.p_h = Q_(7.0)
cc.ionic_strength = Q_("0.15 mM") 
for idx in reactions.index:
    boltzmann_rxn_str = reactions.loc[idx,'Full Rxn']
    if print_output == True: print('Reaction ', idx)
    full_rxn_str_no_cmprt = re.sub(':\S+','', boltzmann_rxn_str)
    if print_output == True:print(full_rxn_str_no_cmprt)
    
    try:
        if (np.issubdtype(reactions.loc[idx,deltag0].dtype, np.number)):
            if print_output == True:print('Skipping...\n')
            continue
    except:
        if print_output == True:print('Calculating...')        
    
    full_rxn_str_no_cmprt = utils.substitute_common_metabolite_names(full_rxn_str_no_cmprt)
    try:
        rxn = cc.search_reaction(full_rxn_str_no_cmprt)
    except ValueError:
        raise
    
    
    if not rxn.is_balanced():
        if print_output == True:print('Reaction %s is not balanced:\n %s\n%s\n' % (idx, full_rxn_str_no_cmprt, rxn), flush=True)
        break
    
    for item in rxn.items(protons=False):
        if print_output == True:print(item[0].get_common_name(), item[1])
        if print_output == True:print(item[0].get_accession())
    output = eq_api.dg_prime(rxn)
    dG0_prime = output.value.m_as("kJ/mol")
    dG0_uncertainty = output.error.m_as("kJ/mol")
    if print_output == True: print(output,'\n')
    
    reactions.loc[idx,deltag0] = float(dG0_prime)
    reactions.loc[idx,deltag0_sigma] = float(dG0_uncertainty)     
    
    
    reactions.loc[idx,deltag0] = float(dG0_prime)
    reactions.loc[idx,deltag0_sigma] = float(dG0_uncertainty)

   

'0.4.7'

## Set Fixed Concentrations/Boundary Conditions

In [48]:
conc = 'Conc'
variable = 'Variable'
metabolites = pd.DataFrame(index = S_active.columns, columns=[conc,variable])
metabolites[conc] = 0.001
metabolites[variable] = True
metab_list1 = metabolites.index

In [49]:
# Set the fixed metabolites:

if "ADENOSINE-3',5'-BISPHOSPHATE:CYTOPLASM" in metabolites.index:
    metabolites.loc["ADENOSINE-3',5'-BISPHOSPHATE:CYTOPLASM",conc] = 1.0e-03
    metabolites.loc["ADENOSINE-3',5'-BISPHOSPHATE:CYTOPLASM",variable] = False 

metabolites.loc['ATP:CYTOPLASM',conc] = 9.600000e-02
metabolites.loc['ATP:CYTOPLASM',variable] = False
metabolites.loc['ADP:CYTOPLASM',conc] = 5.600000e-06
metabolites.loc['ADP:CYTOPLASM',variable] = False

if 'ORTHOPHOSPHATE:CYTOPLASM' in metabolites.index:
    metabolites.loc['ORTHOPHOSPHATE:CYTOPLASM',conc] = 2.000000e-02
    metabolites.loc['ORTHOPHOSPHATE:CYTOPLASM',variable] = False
if 'PHOSPHATE:CYTOPLASM' in metabolites.index:
    metabolites.loc['PHOSPHATE:CYTOPLASM',conc] = 2.000000e-02
    metabolites.loc['PHOSPHATE:CYTOPLASM',variable] = False
if 'AMP:CYTOPLASM' in metabolites.index:
    metabolites.loc['AMP:CYTOPLASM',conc] = 2.81e-07
    metabolites.loc['AMP:CYTOPLASM',variable] = False 
if 'DIPHOSPHATE:CYTOPLASM' in metabolites.index:
    metabolites.loc['DIPHOSPHATE:CYTOPLASM',conc] = 1.0e-04
    metabolites.loc['DIPHOSPHATE:CYTOPLASM',variable] = False 



# High NAD/NADH
metabolites.loc['NADH:CYTOPLASM',conc] = 8.300000e-04
metabolites.loc['NADH:CYTOPLASM',variable] = False
metabolites.loc['NAD+:CYTOPLASM',conc] = 2.600000e-03
metabolites.loc['NAD+:CYTOPLASM',variable] = False


# Low ratio of NADP/NADPH
metabolites.loc['NADPH:CYTOPLASM',conc] = 1.200000e-04
metabolites.loc['NADPH:CYTOPLASM',variable] = False
metabolites.loc['NADP+:CYTOPLASM',conc] = 2.100000e-06
metabolites.loc['NADP+:CYTOPLASM',variable] = False

metabolites.loc['COA:CYTOPLASM',conc] = 1.400000e-06
metabolites.loc['COA:CYTOPLASM',variable] = False

metabolites.loc['CO2:CYTOPLASM',conc] = 1.000e-04
metabolites.loc['CO2:CYTOPLASM',variable] = False 

metabolites.loc['NH3:CYTOPLASM',conc] = 1.000e-04
metabolites.loc['NH3:CYTOPLASM',variable] = False 

metabolites.loc['H2O:CYTOPLASM',conc] = 55.5
metabolites.loc['H2O:CYTOPLASM',variable] = False 
    
if 'A_REDUCED_THIOREDOXIN:CYTOPLASM' in metabolites.index:
    metabolites.loc['A_REDUCED_THIOREDOXIN:CYTOPLASM',conc] = 2.0e-03
    metabolites.loc['A_REDUCED_THIOREDOXIN:CYTOPLASM',variable] = False
if 'AN_OXIDIZED_THIOREDOXIN:CYTOPLASM' in metabolites.index:
    metabolites.loc['AN_OXIDIZED_THIOREDOXIN:CYTOPLASM',conc] = 2.0e-03
    metabolites.loc['AN_OXIDIZED_THIOREDOXIN:CYTOPLASM',variable] = False
if 'A_REDUCED_FERREDOXIN:CYTOPLASM' in metabolites.index:
    metabolites.loc['A_REDUCED_FERREDOXIN:CYTOPLASM',conc] = 2.0e-03
    metabolites.loc['A_REDUCED_FERREDOXIN:CYTOPLASM',variable] = False
if 'AN_OXIDIZED_FERREDOXIN:CYTOPLASM' in metabolites.index:
    metabolites.loc['AN_OXIDIZED_FERREDOXIN:CYTOPLASM',conc] = 2.0e-03
    metabolites.loc['AN_OXIDIZED_FERREDOXIN:CYTOPLASM',variable] = False

''' 
Ethanol concentration from:
North JA, Miller AR, Wildenthal JA, Young SJ, Tabita FR. 
Microbial pathway for anaerobic 5'-methylthioadenosine metabolism coupled to ethylene formation. 
Proc Natl Acad Sci U S A. 2017 Nov 28;114(48)
'''

if 'ETHANOL:CYTOPLASM' in metabolites.index:
    metabolites.loc['ETHANOL:CYTOPLASM',conc] = 85.0e-03 #Lit value
    metabolites.loc['ETHANOL:CYTOPLASM',variable] = False 


if 'ETHYLENE:CYTOPLASM' in metabolites.index:
    metabolites.loc['ETHYLENE:CYTOPLASM',conc] = 1.0e-07
    metabolites.loc['ETHYLENE:CYTOPLASM',variable] = False
    

if 'SULFATE:CYTOPLASM' in metabolites.index:
    metabolites.loc['SULFATE:CYTOPLASM',conc] = 1.0e-02
    metabolites.loc['SULFATE:CYTOPLASM',variable] = False
    

if 'TETRAHYDROFOLATE:CYTOPLASM' in metabolites.index:
    metabolites.loc['TETRAHYDROFOLATE:CYTOPLASM',conc] = 2.0e-03
    metabolites.loc['TETRAHYDROFOLATE:CYTOPLASM',variable] = False

if 'N10-FORMYLTETRAHYDROFOLATE:CYTOPLASM' in metabolites.index:
    metabolites.loc['N10-FORMYLTETRAHYDROFOLATE:CYTOPLASM',conc] = 2.0e-03
    metabolites.loc['N10-FORMYLTETRAHYDROFOLATE:CYTOPLASM',variable] = False

if 'A 7,8-DIHYDROFOLATE:CYTOPLASM' in metabolites.index:
    metabolites.loc['A 7,8-DIHYDROFOLATE:CYTOPLASM',conc] = 2.0e-03
    metabolites.loc['A 7,8-DIHYDROFOLATE:CYTOPLASM',variable] = False
    
metabolites.loc['5-METHYLTETRAHYDROFOLATE:CYTOPLASM',conc] = 2.0e-03
metabolites.loc['5-METHYLTETRAHYDROFOLATE:CYTOPLASM',variable] = False
metabolites.loc['5,10-METHYLENETETRAHYDROFOLATE:CYTOPLASM',conc] = 2.0e-03
metabolites.loc['5,10-METHYLENETETRAHYDROFOLATE:CYTOPLASM',variable] = False


# autoinducer from SAM cycle = (4S)-4,5-DIHYDROXYPENTAN-2,3-DIONE:CYTOPLASM
if '(4S)-4,5-DIHYDROXYPENTAN-2,3-DIONE:CYTOPLASM' in metabolites.index:
    metabolites.loc['(4S)-4,5-DIHYDROXYPENTAN-2,3-DIONE:CYTOPLASM',conc] = 2.0e-09
    metabolites.loc['(4S)-4,5-DIHYDROXYPENTAN-2,3-DIONE:CYTOPLASM',variable] = False


if 'ADENINE:CYTOPLASM' in metabolites.index:
    metabolites.loc['ADENINE:CYTOPLASM',conc] = 2.0e-06
    metabolites.loc['ADENINE:CYTOPLASM',variable] = False
if 'ADENOSINE:CYTOPLASM' in metabolites.index:
    metabolites.loc['ADENOSINE:CYTOPLASM',conc] = 2.0e-14
    metabolites.loc['ADENOSINE:CYTOPLASM',variable] = False


if 'FD_OX:CYTOPLASM' in metabolites.index:
    metabolites.loc['FD_OX:CYTOPLASM',conc] = 2.0e-06
    metabolites.loc['FD_OX:CYTOPLASM',variable] = False
if 'FD_RED:CYTOPLASM' in metabolites.index:
    metabolites.loc['FD_RED:CYTOPLASM',conc] = 2.0e-03
    metabolites.loc['FD_RED:CYTOPLASM',variable] = False
if 'H2:CYTOPLASM' in metabolites.index:
    metabolites.loc['H2:CYTOPLASM',conc] = 2.0e-09
    metabolites.loc['H2:CYTOPLASM',variable] = False



metabolites.sort_values(by=variable, axis=0,ascending=False, inplace=True,)


### Protein Synthesis Reaction

Protein synthesis involves loading the amino acid onto tRNA, the polymerization step where ADP is released
The overall reaction for amino acid polymerization into a polypeptide is,

\begin{array}{lrcll}
\text{tRNA charging:} & \text{AA + tRNA + ATP} & \rightarrow & \text{AMP + AA-tRNA +  PP}_{i} & \\
\text{tRNA loading:} &\text{AA-tRNA + GTP + polyAA}_{n}\text{-tRNA} & \rightarrow & \text{AA-tRNA-polyAA}_{n}\text{-tRNA + GDP + P}_{i} & \\
\text{peptide bond formation:} & \text{AA-tRNA-polyAA}_{n}\text{-tRNA}  & \rightarrow & \text{polyAA}_{n+1}\text{-tRNA + tRNA} & \\
\hline
\text{Overall:} & \text{AA + ATP + GTP} & \rightarrow & \text{polyAA}_{n}\text{-tRNA + AMP + GDP + PP}_{i} + \text{P}_{i} & \\
\end{array}

In this simple scheme, neither the initiation or termination of the translational complex are modeled. 
- Free energy of peptide bond hydrolysis from a protein:

| Reaction | Energy Change |
|:---:|---:|
|  protein-aa $\rightarrow$ protein + aa   |   $\Delta G_{rxn}$ ~ $-0.5$ kcal/mol or ~ $-1.24$ kJ/mol. |
    
From (Jencks, W.P. (1970) in Handbook of Biochemistry (Sober, H.A., ed.), 2nd edn., pp. J181-J186, Chemical Rubber, Cleveland, OH; Watson, J.D. (1976) Molecular Biology of the Gene, 3rd edn., Benjamin, Menlo Park, CA; Are there more recent values?
**See: Martin Biopolymers, Vol. 45, 351–353 (1998)**)

- Free energy of ATP = AMP + diphosphate ~ $-37.1$ kJ/mol.

- Free energy of GTP = GDP + phosphate ~ $-24.4$ kJ/mol.

- Free energy of amino acid polymerization using ATP ~ $-37.1 -24.4 - (-1.2) = -60.3$ kJ/mol.

Then to incorporate the 20 amino acids evenly into a 20-mer peptide would be $20\cdot(-60.3) = -1206$ kJ/mol.

In [50]:


aa_list = ['L-ALANINE:CYTOPLASM','L-ASPARAGINE:CYTOPLASM','L-ASPARTATE:CYTOPLASM','L-ARGININE:CYTOPLASM',\
           'L-CYSTEINE:CYTOPLASM','GLYCINE:CYTOPLASM', \
           'L-GLUTAMATE:CYTOPLASM','L-GLUTAMINE:CYTOPLASM', 'L-HISTIDINE:CYTOPLASM', \
           'L-ISOLEUCINE:CYTOPLASM','L-LEUCINE:CYTOPLASM', \
           'L-LYSINE:CYTOPLASM', 'L-METHIONINE:CYTOPLASM',  \
           'L-PHENYLALANINE:CYTOPLASM','L-PROLINE:CYTOPLASM','L-SERINE:CYTOPLASM',\
           'L-THREONINE:CYTOPLASM','L-TRYPTOPHAN:CYTOPLASM', \
           'L-TYROSINE:CYTOPLASM','L-VALINE:CYTOPLASM']
protein_aa_list = []
for aa in aa_list: 
    if aa in metabolites.index: protein_aa_list.append(aa) 



reactions.loc['protein_syn',left] = 'Amino Acids'
reactions.loc['protein_syn',right] = 'PROTEIN'
reactions.loc['protein_syn',deltag0] = np.array([-60.3])[0]
reactions.loc['protein_syn','ENZYME_LEVEL'] = 1.0 
reactions.loc['protein_syn','Full Rxn'] = 'Amino Acids = Protein' 
reactions.loc['protein_syn','LEFT_COMPARTMENT'] = 'CYTOPLASM'
reactions.loc['protein_syn','RIGHT_COMPARTMENT'] = 'VARIOUS'
reactions.loc['protein_syn','Full Rxn'] = 'Amino Acids = Protein' 
reactions.loc['protein_syn','Same Compartment?'] = 'False' 

num_aa = len(protein_aa_list)
aa_stoichiometry = -1/(num_aa)
S_active.loc[:,'PROTEIN'] = 0;
S_active.loc['protein_syn',:] = 0;
S_active.loc['protein_syn','PROTEIN'] = 1;
for aa in protein_aa_list:
    S_active.loc['protein_syn',aa] = aa_stoichiometry;

metabolites.loc['PROTEIN',conc] = 1.00000e-09
metabolites.loc['PROTEIN',variable] = False

/var/folders/45/hnmrvf5d6wdf1h8m7d4cn5s4hp2pl3/T/ipykernel_26744/3014134948.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  S_active.loc[:,'PROTEIN'] = 0;


### RNA Synthesis

In [51]:



ntp_list = ['ATP:CYTOPLASM','GTP:CYTOPLASM','CTP:CYTOPLASM','TTP:CYTOPLASM']


reactions.loc['rna_syn',left] = 'NTPs'
reactions.loc['rna_syn',right] = 'RNA'
reactions.loc['rna_syn',deltag0] = np.array([-112.02])[0] *0.5 # 1/2 since reaction for single nucleotide instead of a base pair
reactions.loc['rna_syn','ENZYME_LEVEL'] = 1.0 
reactions.loc['rna_syn','Full Rxn'] = 'NTPs = RNA' 
reactions.loc['rna_syn','LEFT_COMPARTMENT'] = 'CYTOSOL'
reactions.loc['rna_syn','RIGHT_COMPARTMENT'] = 'VARIOUS'
reactions.loc['rna_syn','Same Compartment?'] = 'False' 

num_ntp = len(ntp_list)
ntp_stoichiometry = -1/num_ntp # The factor of two is due to 1 dNTP for each strand.
S_active.loc[:,'RNA'] = 0;
S_active.loc['rna_syn',:] = 0;
S_active.loc['rna_syn','RNA'] = 1;
for ntp in ntp_list:
    S_active.loc['rna_syn',ntp] = ntp_stoichiometry;

metabolites.loc['RNA',conc] = 1.00000e-08
metabolites.loc['RNA',variable] = False

/var/folders/45/hnmrvf5d6wdf1h8m7d4cn5s4hp2pl3/T/ipykernel_26744/406738606.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  S_active.loc[:,'RNA'] = 0;
/var/folders/45/hnmrvf5d6wdf1h8m7d4cn5s4hp2pl3/T/ipykernel_26744/406738606.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  S_active.loc['rna_syn',ntp] = ntp_stoichiometry;


### DNA Synthesis

In [52]:





dntp_list = ['DATP:CYTOPLASM','DGTP:CYTOPLASM','DCTP:CYTOPLASM','DTTP:CYTOPLASM']



reactions.loc['dna_syn',left] = 'dNTPs'
reactions.loc['dna_syn',right] = 'DNA'
reactions.loc['dna_syn',deltag0] = np.array([-112.02])[0]    
reactions.loc['dna_syn','ENZYME_LEVEL'] = 1.0 
reactions.loc['dna_syn','Full Rxn'] = 'dNTPs = DNA' 
reactions.loc['dna_syn','LEFT_COMPARTMENT'] = 'CYTOPLASM'
reactions.loc['dna_syn','RIGHT_COMPARTMENT'] = 'VARIOUS'
reactions.loc['dna_syn','Same Compartment?'] = 'False' 

num_dntp = len(dntp_list)
dntp_stoichiometry = -2/num_dntp
S_active.loc[:,'DNA'] = 0;
S_active.loc['dna_syn',:] = 0;
S_active.loc['dna_syn','DNA'] = 1;
for dntp in dntp_list:
    S_active.loc['dna_syn',dntp] = dntp_stoichiometry;

metabolites.loc['DNA',conc] = 1.00000e-09
metabolites.loc['DNA',variable] = False

/var/folders/45/hnmrvf5d6wdf1h8m7d4cn5s4hp2pl3/T/ipykernel_26744/632721797.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  S_active.loc[:,'DNA'] = 0;


## Prepare model for optimization

- Adjust S Matrix to use only reactions with activity > 0, if necessary.
- Water stoichiometry in the stiochiometric matrix needs to be set to zero since water is held constant.
- All concentrations are changed to log counts.
- Equilibrium constants are calculated from standard free energies of reaction.

In [53]:


## Stoichiometric Matrix
active_reactions = reactions[reactions[enzyme_level] > 0.0].copy()
Sactive_index = S_active.index
active_reactions.reindex(index = Sactive_index, copy = True)
S_active = S_active.reindex(columns = metabolites.index, copy = False)
S_active['H2O:CYTOPLASM'] = 0


## Metabolite upper bounds and fixed concentrations
metabolites['Conc'] = np.log(Concentration2Count*metabolites['Conc'])
metabolites = metabolites.rename(columns = {'Conc':'Log_count'})



## Equilibrium Constants
Keq = np.exp(-active_reactions[deltag0].astype('float')/RT)
Keq = Keq.values
K_df = pd.DataFrame(Keq,index = S_active.index, columns = ['Equilibrium Constant'])






In [55]:
### Save model files
S_active.to_csv('Stoichiometric_matrix.csv')
K_df.to_csv('Equilibrium_constants.csv')
metabolites.to_csv('Metabolites.csv')